# Automated ML

In [2]:
import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.core.webservice import Webservice
from azureml.core.webservice import AciWebservice
from azureml.core.model import InferenceConfig
from azureml.core.model import Model

print("SDK version:", azureml.core.VERSION)

SDK version: 1.19.0


## Dataset

### Overview
The dataset chosen for this project is the one from [Kaggle Titanic Challenge](https://www.kaggle.com/c/titanic). 
In the famous Titanic shipwreck, some passengers were more likely to survive than others. The dataset presents information about 871 passengers and a column that states if they have survived or not. The model that we will create using automated ML predicts which passengers survived the Titanic shipwreck.

In [3]:
ws = Workspace.from_config()

# Name of the experiment
experiment_name = 'titanic-automl-experiment'
experiment=Experiment(ws, experiment_name)

# Get the data of Kaggle Titanic Dataset
key = "titanic-modified"
description_text = "Kaggle Titanic Challenge dataset with some changes made by myself"
found = False

if key in ws.datasets.keys(): 
    found = True
    dataset = ws.datasets[key] 

if not found:
    # Create AML Dataset and register it into Workspace
    example_data = 'https://raw.githubusercontent.com/clasimoes/nd00333-capstone/master/titanic_data/full_capstone.csv'
    dataset = Dataset.Tabular.from_delimited_files(example_data)
    #Register Dataset in Workspace
    dataset = dataset.register(workspace=ws,
                               name=key,
                               description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,Q,S,male
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.066409,0.523008,0.381594,32.204208,0.086420,0.725028,0.647587
std,257.353842,0.486592,0.836071,13.244532,1.102743,0.806057,49.693429,0.281141,0.446751,0.477990
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,22.000000,0.000000,0.000000,7.910400,0.000000,0.000000,0.000000
50%,446.000000,0.000000,3.000000,26.000000,0.000000,0.000000,14.454200,0.000000,1.000000,1.000000
75%,668.500000,1.000000,3.000000,37.000000,1.000000,0.000000,31.000000,0.000000,1.000000,1.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200,1.000000,1.000000,1.000000


### Create a compute cluster

We create here a compute cluster to run the experiment. In this cluster, we provise 2-10 machines with the "STANDARD_DS12_V2" configuration.

In [4]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "cluster-2"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS12_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=10)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 2, timeout_in_minutes = 10)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## AutoML Configuration

The task here is to classify if each passenger survived the Titanic shipwreck or not, what led us to provide the column "Survived" as the classification target.
Because we have 10 nodes in our cluster, we are able to run up to 9 concurrent iterations in our experiment (1 node is meant to be used by the "parent" experiment). We have chosen accuracy as the primary metric to be optimized. Featurization is set to auto, leaving data to be automatically scaled and normalized by AutoML. 

In [5]:
# TODO: Put your automl settings here
automl_settings = {"max_concurrent_iterations": 9,
                   "primary_metric" : 'accuracy'
}
project_folder = './capstone-project'

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             experiment_timeout_minutes = 20,
                             training_data=dataset,
                             label_column_name="Survived",   
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings)

In [6]:
# TODO: Submit your experiment
# RemoteRun is an object of type AutoMLRun
remote_run = experiment.submit(automl_config, show_output=False)

Running on remote.


## Run Details

`RunDetails` widget shows the run status per job.

In [7]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

Here get the best model from the automl experiments and display all the properties of the model.



In [9]:
best_run_automl, fitted_model_automl = remote_run.get_output()
model_name = best_run_automl.properties['model_name']

In [10]:
# run
print(best_run_automl)

Run(Experiment: titanic-automl-experiment,
Id: AutoML_a389c44f-f32e-4d8e-93c3-f75565ec41bf_46,
Type: azureml.scriptrun,
Status: Completed)


In [12]:
best_run_automl.get_file_names()

['accuracy_table',
 'automl_driver.py',
 'azureml-logs/55_azureml-execution-tvmps_c4afe6277d5e58e8625f946df952152b33c7dd36d96658430ebe94178484b0e5_d.txt',
 'azureml-logs/65_job_prep-tvmps_c4afe6277d5e58e8625f946df952152b33c7dd36d96658430ebe94178484b0e5_d.txt',
 'azureml-logs/70_driver_log.txt',
 'azureml-logs/75_job_post-tvmps_c4afe6277d5e58e8625f946df952152b33c7dd36d96658430ebe94178484b0e5_d.txt',
 'azureml-logs/process_info.json',
 'azureml-logs/process_status.json',
 'confusion_matrix',
 'logs/azureml/104_azureml.log',
 'logs/azureml/azureml_automl.log',
 'logs/azureml/dataprep/python_span_3d7cab30-11fb-4e9c-a278-8292d9a14195.jsonl',
 'logs/azureml/dataprep/python_span_c755ae9a-6ce5-46b5-910e-9baac2de047f.jsonl',
 'logs/azureml/job_prep_azureml.log',
 'logs/azureml/job_release_azureml.log',
 'outputs/conda_env_v_1_0_0.yml',
 'outputs/env_dependencies.json',
 'outputs/model.pkl',
 'outputs/pipeline_graph.json',
 'outputs/scoring_file_v_1_0_0.py']

In [40]:
best_run_automl.get_metrics()

{'precision_score_macro': 0.8369747044893497,
 'matthews_correlation': 0.6517597038188496,
 'recall_score_micro': 0.8383895131086142,
 'recall_score_macro': 0.8155208543736088,
 'accuracy': 0.8383895131086142,
 'AUC_weighted': 0.8662091324417831,
 'average_precision_score_macro': 0.8581673798861476,
 'f1_score_weighted': 0.8357253888532586,
 'AUC_macro': 0.8662091324417831,
 'precision_score_micro': 0.8383895131086142,
 'weighted_accuracy': 0.8576032369163329,
 'average_precision_score_weighted': 0.8642575782003764,
 'f1_score_macro': 0.8211877676105205,
 'AUC_micro': 0.88315638379616,
 'log_loss': 0.42709466352594283,
 'recall_score_weighted': 0.8383895131086142,
 'f1_score_micro': 0.8383895131086142,
 'norm_macro_recall': 0.6310417087472178,
 'average_precision_score_micro': 0.8649129087688637,
 'precision_score_weighted': 0.8421310933602884,
 'balanced_accuracy': 0.8155208543736088,
 'confusion_matrix': 'aml://artifactId/ExperimentRun/dcid.AutoML_a389c44f-f32e-4d8e-93c3-f75565ec41bf

In [48]:
best_run_automl.properties

{'runTemplate': 'automl_child',
 'pipeline_id': '__AutoML_Ensemble__',
 'pipeline_spec': '{"pipeline_id":"__AutoML_Ensemble__","objects":[{"module":"azureml.train.automl.ensemble","class_name":"Ensemble","spec_class":"sklearn","param_args":[],"param_kwargs":{"automl_settings":"{\'task_type\':\'classification\',\'primary_metric\':\'accuracy\',\'verbosity\':20,\'ensemble_iterations\':15,\'is_timeseries\':False,\'name\':\'titanic-automl-experiment\',\'compute_target\':\'cluster-2\',\'subscription_id\':\'a24a24d5-8d87-4c8a-99b6-91ed2d2df51f\',\'region\':\'southcentralus\',\'spark_service\':None}","ensemble_run_id":"AutoML_a389c44f-f32e-4d8e-93c3-f75565ec41bf_46","experiment_name":"titanic-automl-experiment","workspace_name":"quick-starts-ws-134553","subscription_id":"a24a24d5-8d87-4c8a-99b6-91ed2d2df51f","resource_group_name":"aml-quickstarts-134553"}}]}',
 'training_percent': '100',
 'predicted_cost': None,
 'iteration': '46',
 '_aml_system_scenario_identification': 'Remote.Child',
 '_azu

In [46]:
best_run_automl.properties['run_algorithm']

'VotingEnsemble'

In [47]:
best_run_automl.properties['run_properties']

"classification_labels=None,\n                              estimators=[('0',\n                                           Pipeline(memory=None,\n                                                    steps=[('maxabsscaler',\n                                                            MaxAbsScaler(copy=True"

In [49]:
best_run_automl.properties['fitted_pipeline']

"Pipeline(memory=None,\n         steps=[('datatransformer',\n                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,\n                                 feature_sweeping_config=None,\n                                 feature_sweeping_timeout=None,\n                                 featurization_config=None, force_text_dnn=None,\n                                 is_cross_validation=None,\n                                 is_onnx_compatible=None, logger=None,\n                                 observer=None, task=None, working_dir=None)),\n                ('prefittedsoftvotingclassifier',...\n                                                                                               n_estimators=50,\n                                                                                               n_jobs=1,\n                                                                                               nthread=None,\n                                                    

In [13]:
#Save the best model - download pkl and register model in Azure
model_name = 'best_model_titanic'
model = best_run_automl.register_model(model_name, model_path='outputs/model.pkl')

In [14]:
best_run_automl.download_file('outputs/scoring_file_v_1_0_0.py', 'automl_output/score.py')
best_run_automl.download_file('outputs/model.pkl', 'automl_output/model.pkl')

In [24]:
print(model)

Model(workspace=Workspace.create(name='quick-starts-ws-134553', subscription_id='a24a24d5-8d87-4c8a-99b6-91ed2d2df51f', resource_group='aml-quickstarts-134553'), name=best_model_titanic, id=best_model_titanic:1, version=1, tags={}, properties={})


## Model Deployment

Create an inference config and deploy the model as a web service.

In [15]:
service_name = 'my-titanic-automl-service'
inference_config = InferenceConfig(entry_script='automl_output/score.py')
service = Model.deploy(ws, service_name, [model], inference_config, overwrite=True)
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running...............................................................
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [16]:
print(service.state)

Healthy


Send a request to the web service you deployed to test it.

In [17]:
import json

data = {"data":
        [
          {
            "PassengerId": 812,
            "Pclass": 2,
            "Name": "Berriman, Mr. William John",
            "Age": 23.0,
            "SibSp": 0,
            "Parch": 0, 
            "Ticket": "28425",
            "Fare": 13.0,
            "Cabin": "",
            "Q": 0,
            "S": 1,
            "male": 1
          },
          {
            "PassengerId": 813,
            "Pclass": 1,
            "Name": "Lesurer, Mr. Gustave J",
            "Age": 35.0,
            "SibSp": 0,
            "Parch": 0, 
            "Ticket": "PC 17755",
            "Fare": 512.3292,
            "Cabin": "B101",
            "Q": 0,
            "S": 0,
            "male": 1
          }
      ]
    }

# Convert to JSON string
input_data = json.dumps(data)

Print the logs of the web service and delete the service

In [18]:
output = service.run(input_data)

In [19]:
print(output)

{"result": [0.0, 1.0]}


Delete the service:

In [23]:
service.delete()

In [25]:
print(service.state)

Deleting
